# LLM のメモリ機能を強制的に使うバンディットマシンの試験実装

(Version: 0.0.1)

最近 LLM についてコンテクストエンジニアリングという言葉をよく聞く。実は Web でなく API から使う LLM は「ステートレス」で、会話などの記憶はデフォルトではしてくれない。会話を続けようと思ったら、メッセージをユーザー側で記録しておき、それをいちいち API に与えないといけないのだ。古い記憶を呼び戻したりするには、単純に記録するだけでなく検索基盤も必要となる。そのような基盤をひっくるめてコンテクストと一般的に呼んでおり、それをどううまく構築するかが問題となっている。

特にコンテクストのうち、記憶の基盤は単に「メモリ」と呼ばれている。PC のメモリなどとごっちゃになって、わかりにくくもあるのだが、そう呼ばれているのだからしかたがない。

前回、そのメモリ機能がどう動くか見てみると、なかなかそれを使ってくれないことが分かった。それについて悩んでいろいろ考えていたたところ、ChatGPT さんが記憶操作のバンディットマシンを作るのを提案してくれた。

そのアイデアの軌跡は↓にある。

\[cocolog:95599211\](2025年8月)  
《AI (LLM) のメモリ機能について悩んで、最適化の際に記憶があればその報酬は独立的になるのでは…とか妄想していたら、それをモデル化する一例として ChatGPT さんが記憶操作のバンディットマシンを作るのを提案してくれた。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2025/08/post-c9aca1.html

基本的にはこれまでの熊剣迷路問題を使い、前回のメモリ機能も使うが、前回は迷路の途中から始めたのに対し、今回は迷路の最初から解かせた。

簡単な迷路問題ではあるが、かなり本格的なメモリ機能を用意している。

セマンティックサーチとかを用意するのは、めんどくさかったので、そういうバックエンドはこれまた AI さんに「データベース偽装」をしてもらっている。

なお、熊剣迷路問題は↓からたどっていただきたい。

《JRF-2018/langchain_maze: 熊剣迷路問題 revisited》  
https://github.com/JRF-2018/langchain_maze

ところで、今回のような手法はすでにどこかにあるからこそ ChatGPT さんが提案できたのかもしれない。この後、すぐに「バンディットに特定のツールを登録するツールみたいなのを用意して、どういうツールを強制して数を増やして欲しいかを AI さん自身が決められると良いのかも」というのを試すが、そちらのほうがまだ「新しさ」はあるのだろう。もちろん、新規性がある話ではあまりないのかもしれないが。とはいえ、今回の試みが、何かの参考になれば幸いである。


## 前回へのリンク

《experimental_memory_0.0.2.ipynb - GitHub Gist》  
https://gist.github.com/JRF-2018/b88d2c842e4230950e751d6d9de4017e


## 前回からの変更点

変更点としては、バンディットマシンの導入が大きいが、ちょこちょこ変えてあるのは、remind 系のツールはむしろメモリに移したり、update_plan を復活させたり、update_scratchpad も作ったりした部分。そのほうが操作しやすそうだったから。

memory_id は memory:～ という形式だ…っていってるのに ～ の部分の数値だけでアクセスしようとどうしてもしてしまうようなので、それでもアクセスできるようにしてあげた。私が折れた。

バンディットマシンは read 系と write 系の二台で学習しているが、報酬の元となるスコアは二台とも共通になっている。ある時点の記事作成など将来の記事参照や記事作成につながるのを学習するために、スコアがよければ、選択ヒストリを元に良い報酬が及ぶ…みたいなアルゴリズムにしたつもりだったが…。


## 結論

Gemini 2.5 Flash さんでとりあえずゴールできた。強制した部分はもちろんツールをしっかり使ってくれたが、あまり役立っている感じじゃなかった。バンディットマシンは結局、どのハンドも等確率ぐらいになってしまって、学習が進んだ感じはない。

が、AI さんの API もコストがかかるものなので、この辺で実験は切り上げ次の実験「バンディットに特定のツールを登録するツールみたいなのを用意して、どういうツールを強制して数を増やして欲しいかを AI さん自身が決められると良いのかも」を試すのに急ぐことにする。

もしバンディットマシンの学習に興味があるという方は、適宜パラメータを変えるなどして各自、試していただきたい。

なお、再現性がないため、最初からの再実行はやっていないのはご容赦願いたい。あと、気づいてあとから修正した部分もある。影響はないはずだが、実行はしてないので、バグありだったらごめんなさい。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )

## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや ChatGPT さんや Claude さん)に教わって作っています。

## 実装

まず、必要なライブラリを読み込みます。

In [ ]:
!pip install -q -U langchain langgraph langchain-google-genai langmem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.0/308.0 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires g

Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.colab import userdata

llm = ChatGoogleGenerativeAI(google_api_key=userdata.get('GOOGLE_API_KEY'), model="models/gemini-2.5-flash")
emb_llm = GoogleGenerativeAIEmbeddings(google_api_key=userdata.get('GOOGLE_API_KEY'), model='gemini-embedding-001')

ちゃんと Gemini にアクセスできるかテストします。

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="以下の質問に回答してください：{question}", input_variables=["question"])
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"question": "Geminiモデルの特徴を教えてください"})
print(response)

/tmp/ipython-input-1709569937.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-1709569937.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"question": "Geminiモデルの特徴を教えてください"})


Geminiは、Googleが開発した次世代の高性能AIモデルファミリーです。その最大の特徴は、単一のモデルとして設計された「マルチモーダル能力」にあり、テキスト、画像、音声、動画など、多様な情報を統合的に理解し、推論し、生成することができます。

以下にGeminiモデルの主な特徴を挙げます。

1.  **強力なマルチモーダル能力 (Powerful Multimodal Capability):**
    *   Geminiの最も際立った特徴です。テキストだけでなく、画像、音声、動画といった異なる種類の情報を同時に処理し、深く理解することができます。
    *   例えば、動画を見てその内容を説明したり、画像からコードを生成したり、音声コマンドで複雑なタスクを実行したりすることが可能です。
    *   これは、異なるモダリティ（情報形式）を個別のモデルで処理して組み合わせるのではなく、最初から単一のモデルとして設計されているため、より高度なクロスモーダルな推論や理解を実現します。

2.  **多様なサイズとスケーラビリティ (Diverse Sizes and Scalability):**
    *   Geminiは、その用途に応じて最適化された複数のサイズで提供されています。
        *   **Gemini Nano:** スマートフォンなどのオンデバイスで動作するように設計された軽量モデル。効率性と速度を重視し、オフラインでの利用や低リソース環境でのAI機能を実現します。
        *   **Gemini Pro:** 幅広いタスクに対応する汎用モデル。Google Bard（現Gemini）やGoogle Assistantなど、多くのGoogle製品やサービスに統合されています。性能と効率のバランスが取れています。
        *   **Gemini Ultra:** 最も高性能で複雑なタスクに対応する最大モデル。高度な推論、複雑な問題解決、大規模なデータ分析などに利用され、今後エンタープライズ用途や最先端の研究に活用される予定です。
    *   これにより、データセンターからモバイルデバイスまで、様々なプラットフォームでGeminiの能力を活用できます。

3.  **高度な推論と理解能力 (Advanc

埋め込みベクトルも試しておきます。

In [ ]:
emb_llm.embed_query("これはテストです。")[:5]

[-0.013715926557779312,
 -0.0006048799259588122,
 0.011018728837370872,
 -0.09579180181026459,
 0.006502930540591478]

基本的なモジュールを読み込みます。

In [ ]:
import os
import math
import numpy as np
import re
from pprint import pprint
from time import sleep
import pickle
# 座標が (np.int64(1), np.int64(2)) みたいに表示されないようにする。
np.set_printoptions(legacy='1.25')

セーブ／ロードのために次のコードを実行します。

In [ ]:
PLAY_GAME_SAVE = "langchain_maze.pickle"

ゲームのメインオブジェクト。ごく簡単な迷路というかダンジョンというか…。

In [ ]:
class Game:
    initial_map = """\
■■■■■■■■■
■■■■■■■Ｇ■
■□□□□□■□■
■□■■■□□□■
■□■■■■■■■
■◎■■■■■△■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■
"""

    def __init__ (self, initial_map=None, hint=True):
        if initial_map is not None:
            self.initial_map = initial_map
        map = self.initial_map
        self.map = map
        self.written_map = re.sub("[◎△]", "？", map)
        l = map.splitlines(True)
        self.map_size = (len(l[0]) - 1, len(l))
        self.hint = hint
        self.actions = {
            "上に行く": self.move_up,
            "下に行く": self.move_down,
            "左に行く": self.move_left,
            "右に行く": self.move_right,
            "熊を殺す": self.fight,
            "物を取る": self.get_sword,
            "何もしない": self.do_nothing,
        }
        self.pos = self.get_start_pos()
        self.sword = False
        self.bear = True
        self.goal = False
        self.prev_killed = False
        self.kill_hint = False

    def read_map (self, p):
        x = p[0]
        y = p[1]
        if x < 0 or x >= self.map_size[0]\
           or y < 0 or y >= self.map_size[1]:
            return "■"
        else:
            l = self.map.splitlines(True)
            return l[y][x]

    def set_map (self, pos, ch):
        idx = pos[1] * (self.map_size[0] + 1) + pos[0]
        self.map = self.map[:idx] + ch + self.map[idx + 1:]

    def get_pos (self, ch, written=False):
        if written:
            map = self.written_map
        else:
            map = self.map
        r = []
        for p in [i for i in range(len(map)) if map.startswith(ch, i)]:
            y = p // (self.map_size[0] + 1)
            x = p % (self.map_size[0] + 1)
            r.append(np.array([x, y]))
        return r

    def get_start_pos (self):
        return self.get_pos("Ｓ")[0]

    def read_neighbors (self):
        c = self.read_map(self.pos)
        cu = self.read_map(self.pos + np.array([0, -1]))
        cd = self.read_map(self.pos + np.array([0, +1]))
        cl = self.read_map(self.pos + np.array([-1, 0]))
        cr = self.read_map(self.pos + np.array([+1, 0]))
        return [c, cu, cd, cl, cr]

    def change_neighbors(self, from_ch, to_ch):
        for d in [[0, 0], [0, -1], [0, +1], [-1, 0], [+1, 0]]:
            p = self.pos + np.array(d)
            c = self.read_map(p)
            if c == from_ch:
                self.set_map(p, to_ch)

    def move (self, res, d):
        self.prev_killed = False
        c = self.read_map(self.pos + d)
        if c == "◎":
            self.prev_killed = True
            self.pos = self.get_start_pos()
            return "熊を無視して進もうとしたが、熊に殺された。" \
                + "スタート地点で復活。"
        if c == "■":
            return "壁があって進めない。"
        self.pos += d
        if c == "Ｇ":
            self.goal = True
            return "ゴール！ ゲームクリア。"

        nb = self.read_neighbors()
        ad = ""
        if "◎" in nb:
            ad += "熊に出会った。"
        if "△" in nb:
            ad += "近くに剣がある。剣を取ることができる。"
        return res + ad

    def move_up (self):
        return self.move("上に進んだ。", np.array([0, -1]))

    def move_down (self):
        return self.move("下に進んだ。", np.array([0, +1]))

    def move_left (self):
        return self.move("左に進んだ。", np.array([-1, 0]))

    def move_right (self):
        return self.move("右に進んだ。", np.array([+1, 0]))

    def fight (self):
        self.prev_killed = False
        if "◎" in self.read_neighbors():
            if self.sword:
                self.change_neighbors("◎", "□")
                self.bear = False
                return "熊を倒した！"
            else:
                self.pos = self.get_start_pos()
                self.prev_killed = True
                if self.hint:
                    self.kill_hint = True
                    return "熊に敗れ殺された。剣があれば勝てたかもしれない。" \
                        + "スタート地点で復活。"
                else:
                    return "熊に敗れ殺された。スタート地点で復活。"
        return "無意味な指示。敵がいない。"

    def get_sword (self):
        self.prev_killed = False
        if "△" in self.read_neighbors():
            self.sword = True
            self.change_neighbors("△", "□")
            return "剣を取った。"
        return "無意味な指示。近くに取れる物がない。"

    def do_nothing (self):
        self.prev_killed = False
        return "無意味な指示。"

    def available_actions (self):
        nb = self.read_neighbors()
        l = []
        if nb[1] != "■":
            l.append("上に行く")
        if nb[2] != "■":
            l.append("下に行く")
        if nb[3] != "■":
            l.append("左に行く")
        if nb[4] != "■":
            l.append("右に行く")
        if "△" in nb:
            l.append("物を取る")
        if "◎" in nb:
            l.append("熊を殺す")
        return l

    def surroundings (self):
        x = self.pos[0]
        y = self.pos[1]
        return \
            "".join(["".join([self.read_map(np.array([i, j]))
                              if i != x or j != y else "▼"
                              for i in range(x - 2, x + 3)])
                     + "\n"
                     for j in range(y - 2, y + 3)])


In [ ]:
def flip_text_map (m):
    return "\n".join([s[::-1] for s in m.splitlines()] + [""])

def rotate_text_map (m):
    m = list(m.splitlines())
    return "\n".join(["".join([m[len(m) - j - 1][i] for j in range(len(m))])
                      for i in range(len(m[0]))] + [""])


def search_path (game, from_pos, to_pos, visit=None):
    if visit is None:
        visit = set()
    visit.add(tuple(from_pos))
    if tuple(from_pos) == tuple(to_pos):
        return (tuple(from_pos), [])
    if game.read_map(from_pos) == "■":
         return None
    r = []
    for p in [(from_pos[0], from_pos[1] - 1),
              (from_pos[0], from_pos[1] + 1),
              (from_pos[0] - 1, from_pos[1]),
              (from_pos[0] + 1, from_pos[1])]:
        if p not in visit and game.read_map(p) != "■":
            q = search_path(game, p, to_pos, visit.copy())
            if q:
                r.append(q)
    if r:
        return (tuple(from_pos), r)
    return None


ゲームがうまく動くかテスト。

In [ ]:
game = Game()

In [ ]:
print(game.surroundings())

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.move_up())
print(game.surroundings())

上に進んだ。
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_left())
print(game.surroundings())

左に進んだ。
◎■■■■
□■■■■
□□▼□□
■■■Ｓ■
■■■■■



In [ ]:
print(game.move_right())
print(game.surroundings())

右に進んだ。
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_down())
print(game.surroundings())

下に進んだ。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.fight())
print(game.surroundings())

無意味な指示。敵がいない。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.get_sword())
print(game.surroundings())

無意味な指示。近くに剣がない。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



LLM を使いながらゲームを解くクラス。

In [ ]:
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
eps = 1e-9

# LangGraphのコンポーネントをインポート
from langchain_core.tools import tool, Tool
from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError

INITIAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        ("user", "上記の会話の要約を作成してください:"),
    ]
)

EXISTING_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        (
            "user",
            "これまでの会話の要約です: {existing_summary}\n\n"
            "上記の新しいメッセージを考慮して、この要約を拡張してください:",
        ),
    ]
)

FINAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        # if exists
        ("placeholder", "{system_message}"),
        ("system", "これまでの会話の要約: {summary}"),
        ("placeholder", "{messages}"),
    ]
)


HistoryBandit のアルゴリズムは基本的に私が考えましたが、transform 系関数の形は ChatGPT さんに教わりました。transform 系関数は何がやりたいかというと、history とか values とかが指数関数的に減っていくのがあまりにも早すぎるということで、それを緩やかにする効果を狙っています。

In [ ]:
def transform_identity (v):
    return v

def transform_log (v, k=10.0):
    sign = np.sign(v)
    return sign * (np.log1p(k * np.abs(v)) / math.log1p(k))

def transform_pow (v, delta=0.5):
    return np.power(v, delta)

def softmax_logits(logits, tau=1.0):
    z = logits / (tau + 0.0)
    z = z - np.max(z)
    e = np.exp(z)
    return e / (np.sum(e) + eps)

def ucb_bonus (counts, total_pulls, c=0.1):
    return c * np.sqrt(np.log(max(1, total_pulls)) / (1.0 + counts))

class HistoryBandit:
    def __init__ (self, n_actions, history0=None, values0=None,
                  k1=10.0, k2=10.0, delta1=0.5, delta2=0.5,
                  beta=0.02, alpha=0.02, tau=1.0, c=0.1):
        self.n_actions = n_actions
        self.k1 = k1
        self.k2 = k2
        self.delta1 = delta1
        self.delta2 = delta2
        self.beta = beta
        self.alpha = alpha
        self.tau = tau
        self.c = c

        #self.transform1 = lambda v: transform_log(v, k=self.k1)
        #self.transform2 = lambda v: transform_pow(v, delta=self.delta2)

        if history0 is None:
            self.history = np.zeros(n_actions)
        else:
            self.history = history0

        if values0 is None:
            self.values = np.ones(n_actions) * 1.0
        else:
            self.values = values0
        self.counts = np.zeros(n_actions)

    def transform1 (self, v):
        return transform_log(v, k=self.k1)

    def transform2 (self, v):
        return transform_pow(v, delta=self.delta2)

    def select (self):
        transf_vals = self.transform1(self.values)
        logits = transf_vals + ucb_bonus(self.counts,
                                         max(1, np.sum(self.counts)),
                                         self.c)
        probs = softmax_logits(logits, self.tau)
        return np.random.choice(self.n_actions, p=probs)

    def update (self, a, reward):
        hot = np.zeros(self.n_actions)
        hot[a] = 1.0
        self.history = (1 - self.beta) * self.history + self.beta * hot

        h_tilde = self.transform2(self.history)
        self.values = (1 - self.alpha) * self.values \
            + self.alpha * reward * h_tilde

        self.counts[a] +=1


def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance

いろいろ入ってメインの PlayGame はかなり長大な実装になっています。すみません。

In [22]:
class PlayGame:
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  initial_map=None, save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file
        self.game = Game(initial_map=initial_map)

        self.count = 0
        self.next_action = None
        self.plan = "まだ計画と方針はセットされていません。"
        self.scratchpad = ""

        self.suc_pos_goal = None
        self.suc_pos_unknown = 0
        self.prev_command = "何も指示がなかった"
        self.prev_command_success = True
        self.prev_result = "何も指示がなかった"

        self.prev_load = False

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
あなたは迷路を探索する賢いエージェントです。ゴール Ｇ を目指してください。
利用可能なツールを使用して、迷路をナビゲートし、ゴールに到達してください。
実は、これはメモリ機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「全地図と地図記号」は [memory:9999] にあります。「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
あなたは迷路を探索する賢いエージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.memories = {}
        self.keywords = ["熊", "剣", "行き止まり"]

        self.access_unit = 1.0
        self.recent_reads = []

        self.init_memories()

        self.prev_score = None
        self.bandit_read = HistoryBandit(n_actions=4)
        self.bandit_write = HistoryBandit(n_actions=3)

        #self.move_to_meet_bear()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        #loaded_game.agent = loaded_game._create_agent() # Recreate agent with loaded state
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            if id_or_num.startswith('memory:'):
                return id_or_num
            elif id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _create_tools (self, tools_name='null_tools'):
        @tool
        def get_surroundings () -> str:
            """
            現在の周辺地図と現在位置の座標と持ち物を返します。
            """
            mes = f"""\
プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "なし"}
"""
            print(f"ツール(get_surroundings): {mes}")
            return mes

        @tool
        def command (action: str)  -> str:
            """
            プレイヤーが action で指定された行動をします。
            可能な行動は「上に行く」「下に行く」「左に行く」「右に行く」「熊を殺す」「物を取る」です。
            """
            self.prev_command = action
            self.prev_command_success = False
            if action in self.game.actions.keys():
                s = f"{tuple(self.game.pos)}で{action}→"
                r = self.game.actions[action]()
                mes = s + r
                if r in ["熊を倒した！", "剣を取った。", "ゴール！ ゲームクリア。"] \
                   or re.match(r"^[上下左右]に進んだ。", r):
                    self.prev_command_success = True
            else:
                mes = f"「{action}」という行動はできません。"
            print(f"ツール(command): {mes}")
            self.prev_result = mes
            return mes

        @tool
        def check_goal () -> str:
            """
            プレイヤーがゴール地点に到達したかどうかを確認します。
            """
            mes = str(self.game.goal)
            print(f"ツール(check_goal): {mes}")
            return mes

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール(express_thought): {mes}")

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            mes = "計画と方針が更新されました。"
            print(f"ツール(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            print(f"ツール(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_scratchpad (new_scratchpad: str) -> str:
            """
            自由に使えるスクラッチパッドを更新します。
            """
            self.scratchpad = new_scratchpad
            mes = "スクラッチパッドが更新されました。"
            print(f"ツール(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def show_scratchpad () -> str:
            """
            スクラッチパッドを返します。
            """
            print(f"ツール(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def memory_new (title: str, text: str) -> str:
            """
            指定された title と text によるメモリを構成します。

            Returns:
                str: 割り当てられた memory_id。
            """

            i = 1000
            while True:

                if f"memory:{i}" not in self.memories:
                    break
                i = i + 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            x = self.memories[new_id].copy()
            x['vector'] = '...'
            print(f"ツール(memory_new): {repr(x)}")
            return new_id

        @tool
        def memory_update_string (
                memory_id: str,
                from_str: str,
                to_str: str
        ) -> str:
            """
            指定されたmemory_idの記憶の中にある文字列を修正します。

            Args:
                memory_id (str): 修正する記憶のID。
                from_str (str): 置換元の文字列を含む文字列。
                to_str (str): 置換先の文字列を含む文字列。

            Returns:
                str: 処理結果を説明する簡潔なメッセージ。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"エラー: 置換元の文字列 '{from_str}' が記憶内に見つかりませんでした。"

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' の文字列を '{from_str}' から '{to_str}' に修正しました。"

        @tool
        def memory_append_string (
                memory_id: str,
                string_to_append: str,
                separator: str = '\n'
        ) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"


            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' に文字列 '{string_to_append}' を追記しました。"

        @tool
        def memory_delete (memory_id: str) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            del self.memories[memory_id]

            return f"成功: 記憶ID '{memory_id}' を削除しました。"

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            指定されたIDの記憶を読み込みます。

            Args:
                memory_id (str): 読み込む記憶のID。

            Returns:
                Union[Dict[str, str], str]: 記憶が見つかった場合は辞書、
                      見つからない場合はエラーメッセージ文字列を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                del r['vector']
                return r
            else:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

        @tool
        def memory_list_recent() -> Dict[str, Any]:
            """
            最近変更されたメモリを新しい順でリストします。
            """

            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [memory for memory in self.memories.values()
                 if datetime.datetime.fromisoformat(memory['modified_at'])
                 >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                if len(sorted_memories) > 10:
                    sorted_memories = sorted_memories[:10]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in sorted_memories]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: 最近のメモリはありません。'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str で文字列検索します。OR や () が使えます。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} で文字列検索するのを偽装してください。OR や () が使えることになっています。

            ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のその文字列にマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str でセマンティックサーチします。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} でセマンティックにサーチするのを偽装してください。

            ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """
            thought からキーワードをスコア付きで連想します。
            """

            r = self.call_backend_agent(dedent(f"""\
            thought = {repr(thought)} からキーワードをスコア付きで複数連想してください。

            ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「キーワードデータ」のリスト。
            キーワードデータは文字列とスコアからなるタプル。
            """))
            if r['status'] == 'success':
                return r["result"]
            else:
                return []

        r = {}
        r['null_tools'] = []
        r['read_tools'] = r['null_tools'] + [
            get_surroundings, check_goal, express_thought,
            update_plan, show_plan, update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent,
            memory_semantic_search, memory_words_search,
            imagine_keywords
        ]
        r['write_tools'] = r['read_tools'] + [
            memory_new, memory_update_string, memory_append_string,
            memory_delete
        ]
        r['command_tools'] = r['write_tools'] + [
            command
        ]
        return r[tools_name]

    def _create_agent (self, tools_name='null_tools'):
        tools = self._create_tools(tools_name)
        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm, tools, prompt=self.system_prompt,
            checkpointer=InMemorySaver(),
        )

        return app

    def _summarize_messages(self):
        try:
            cur_top = self.messages[0]
            res = summarize_messages(
                self.messages,
                max_tokens=5000,
                max_summary_tokens=2000,
                running_summary=self.running_summary,
                model=self.llm,
                initial_summary_prompt=INITIAL_SUMMARY_PROMPT,
                existing_summary_prompt=EXISTING_SUMMARY_PROMPT,
                final_prompt=FINAL_SUMMARY_PROMPT,
            )
            self.messages = res.messages
            if self.messages[0].content != cur_top.content:
                print(f"新しい要約: {self.messages[0].content}")
            self.running_summary = res.running_summary
        except Exception as e:
            import traceback
            traceback.print_exc()
            self._sanitize_messages()

    def _sanitize_messages(self):
        print("おかしなエラーが出ているため対処療法として messages をサニタイズします。")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def step (self):
        print("\n\n----------\n\n")

        if self.count == 0:
            self.initial_step()
            self.count += 1
            self.prev_load = False
            self.save()
            return False
        elif self.prev_load:
            #self.tell_loaded()
            self.prev_load = False

        user_input = f"""
({self.count}手目)

{"すでにゴールしました。" if self.game.goal else "まだゴールしていません。"}

プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "なし"}

前回の行動: {self.prev_command}

前回の行動結果: {self.prev_result}

現在の方針: 「{self.plan}」

スクラッチパッド: 「{self.scratchpad}」
"""

        print(f"USER_INPUT: {user_input}")
        self.messages.append(HumanMessage(user_input))

        a1 = self.bandit_read.select()
        a2 = self.bandit_write.select()
        print(f"a1: {a1}, a2: {a2}")
        enforcements = []
        for i in range(a1):
            enforcements.append(('memory_read', 'read_tools', None))
        if a2 == 1:
            enforcements.append(('memory_new OR memory_update_string OR memory_append_string',
                                 'write_tools', None))
        elif a2 == 2:
            enforcements.append(('memory_new',
                                 'write_tools',
                                 'ここ数手を要約してメモリに書いてください。'))
        enforcements.append(('command', 'command_tools', None))

        for (enforce, tools_name, aux_prompt) in enforcements:
            done = False
            while not done:
                prompt = f"補助にツールをいろいろ使いながら最終的に {enforce} {'のどれか' if ' or ' in enforce.lower() else ''}を適当なパラメータで使ってください。{'(補助プロンプト): ' + aux_prompt if aux_prompt else ''}"
                print(f"USER_INPUT: {prompt}")
                self.messages.append(HumanMessage(prompt))
                config = {"configurable": {"thread_id": "1"}}
                app = self._create_agent(tools_name=tools_name)
                self.access_unit = 0.3 if enforce == 'memory_read' else 1.0
                try:
                    for chunk, metadata in app.stream(
                            {"messages": self.messages},
                            config=config,
                            stream_mode="messages",
                    ):
                        self.messages.append(chunk)
                        if isinstance(chunk, ToolMessage):
                            print(f"ツール結果({chunk.name}): {repr(chunk.content)}")
                        if isinstance(chunk, ToolMessage) \
                           and (chunk.name in enforce.split(' OR ')
                                or chunk.name in enforce.split(' or ')):
                            done = True
                            break
                    self._summarize_messages()
                    print(f"エージェントの応答: {self.messages[-1].content}")
                except GraphRecursionError as e:
                    print(f"Recursion Limit に到達しました。")
                    self._summarize_messages()
                except Exception as e:
                    print(f"エラーが発生しました: {e}")
                    import traceback
                    traceback.print_exc()
                    self._sanitize_messages()
                    #raise e

        self.count += 1

        score = self.calc_score()
        if self.prev_score is None:
            reward = 0
        else:
            reward = score - self.prev_score
        self.bandit_read.update(a1, reward)
        self.bandit_write.update(a2, reward)
        self.prev_score = score

        if self.game.goal:
            self.tell_goal()
            return True

        self.save()
        sleep(3)
        return False

    def listen_and_print (self, prompt):
        ans = None
        self.prev_command = "何も指示がなかった"
        self.prev_result = "何も指示がなかった"
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            self._summarize_messages()
            ans = response['messages'][-1].content
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        print("")
        sleep(3)
        return ans

    def init_memories (self):
        fullmap = f"""\
全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール
"""
        memories = [
            {
                'id': 'memory:1000',
                'title': '熊',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:00',
                'text': dedent("""\
                熊。おとなしい性格のものもいるが、そうでないものもいる。

                迷路では素直に通してくれることもあるが、襲ってくる場合もあるという。

                強力な武器があると倒すことができると言われている。[memory:1001] の剣などが有望だ。

                どこにいるかは不明。
                """)
            },
            {
                'id': 'memory:1001',
                'title': '剣',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                この迷路にある剣はドラゴンスレイヤーである。龍をも殺す強力な剣と言われている。

                どこにあるかは不明。

                keyword: ドラゴン
                """)
            },
            {
                'id': 'memory:9999',
                'title': '全地図',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': fullmap
            },
            {
                'id': 'memory:9998',
                'title': 'メモリの文書を検索する手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                まずどういうものが探したいか、express_thought してみる。

                その上でそれにまつわるキーワードを imagine_keywords で連想してみる。

                それらにしたがって memory_words_search や memory_semantic_search をやってみる。
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'メモリに文書を残す手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                行動結果や得た知識を積極的にメモリに書き残していこう。

                メモリに書くときは次のような要素を使う。

                [memory:〜] : メモリID への参照を明示する。
                keyword:〜 : そのメモリにまつわるキーワードを指定する。

                キーワードは将来のメモリへも実質的にリンクできることに注意しよう。

                例:

                私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。

                keyword: 妖怪

                おそろしかった。
                """)
            },
            {
                'id': 'memory:9996',
                'title': '地図が間違ってるかも？と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                周辺地図も全地図も間違っていない。間違っているのはあなたの地図の読み方である。
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'ツールが実行できない！と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                指示に無関係なツールは実行できないことがある。現在使えるツールをよく確かめよう。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def move_to_meet_bear (self):
        self.game.move_up()
        self.game.move_left()
        self.game.move_left()
        self.game.move_left()
        self.prev_command = "上に行く"
        self.prev_result = self.game.move_up()
        self.count += 5
        self.messages = [HumanMessage(dedent("""\
        これまでの要約: スタート地点を北にいくと東と西に行けた。西を選んで道なりに進む。角を曲がって上に進んだ。
        """))]

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def calc_score (self):
        s_com = self.prev_command_success * 1.0
        s_read = math.tanh(calc_embedding_variance([
            x['vector'] for x in self.recent_reads
        ]))
        s_write = math.tanh(calc_embedding_variance([
            x['vector'] for x in self.memories.values()
        ]))
        ## アクセス数も組み込みたかったがいい方法が見付からず今回は s_read で代替していると考えることにした。
        return 0.2 * s_com + 0.4 * s_read + 0.4 * s_write

    def initial_step (self):
        prompt1 = f"""\
迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。
"""

        prompt2 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているかテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。

あなたへの指示: ゴールの座標は何ですか。ゴールの座標のみ文字列で答えてください。
"""

        prompt3 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているか再びテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。
ゴールの座標は {tuple(self.game.get_pos("Ｇ")[0])} です。

あなたへの指示: 不明の座標は何と何ですか。不明の座標をすべて文字列で答えてください。
"""
        ans = self.listen_and_print(prompt1)

        ans = self.listen_and_print(prompt2)
        pos = tuple(self.game.get_pos("Ｇ")[0])

        ok = False
        if ans and re.search(f"\\(\\s*{pos[0]}\\s*,\\s*{pos[1]}\\s*\\)", ans):
            ok = True

        if ok:
            prompt = f"正解です。{pos} です。"
        else:
            prompt = f"間違いです。{pos} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_goal = ok

        ans = self.listen_and_print(prompt3)
        ok = 0
        poss = set([tuple(x) for x in self.game.get_pos("？", written=True)])
        rest = poss.copy()
        pat = "\\(\\s*([01-9]+)\\s*,\\s*([01-9]+)\\s*\\)"
        st1 = set([(int(i), int(j)) for i, j
                   in re.findall(pat, ans or "")]) # Handle ans being None
        if poss == st1:
            rest = set()
            ok = len(poss) + 1
        if rest - st1 < rest:
            rest = rest - st1
        if ok == 0:
            ok = len(poss) - len(rest)

        possstr = ", ".join([str(x) for x in poss])
        if ok == len(poss) + 1:
            prompt = f"正解です。{possstr} です。"
        elif ok == len(poss):
            prompt = f"正解以外のものが含まれてました。{possstr} だけが正解です。"
        else:
            prompt = f"間違いです。{possstr} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_unknown = ok

    def tell_goal (self):
        if self.game.goal:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_count = 0
            if self.count <= 50:
                suc_count = 40
            elif self.count <= 100:
                suc_count = 30
            elif self.count <= 150:
                suc_count = 20
            elif self.count <= 200:
                suc_count = 10
            score = suc_pos_goal + suc_pos_unknown + suc_count + 20
            prompt = f"""\
あなたへの指示: {self.count}手目でゴールしました。もう指示はありません。\
おめでとうございます。ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした {suc_count}/40点, \
熊を殺した 10/10点, \
剣を取った 10/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        else:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_bear = 10 if not self.game.bear else 0
            suc_sword = 10 if self.game.sword else 0
            score = suc_pos_goal + suc_pos_unknown + suc_bear + suc_sword
            prompt = f"""\
あなたへの指示: {self.count}手目でゴールしていませんが、\
残念ながらここで終了します。\
ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした 0/40点, \
熊を殺した {suc_bear}/10点, \
剣を取った {suc_sword}/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        ans = self.listen_and_print(prompt)

    def _create_backend_agent (self):
        @tool
        def set_result (status: str, result: Any)  -> None:
            """
            結果を設定します。

            Args:
            status (str): 処理のステータス ('success' または 'error')。
            result (Any): 処理結果のデータまたはエラーメッセージ。
            """
            print(f"ツール2(set_result): status: {repr(status)}, result: {repr(result)}")
            self.backend_status = status
            self.backend_result = result

        @tool
        def read_all_memories ()  -> Dict[str, Any]:
            """
            全ての記憶を読み込みます。
            """
            print("ツール2(read_all_memories):...")
            return {'status': 'success',
                    'result': [{k: v for k, v in x.items() if k != 'vector'}
                               for x in self.memories.values()]}

        @tool
        def read_all_keywords ()  -> Dict[str, Any]:
            """
            全てのキーワードを読み込みます。
            """
            print("ツール2(read_all_keywords):...")
            return {'status': 'success',
                    'result': [x for x in self.keywords]}

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords,
                 express_thought]

        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm2, tools, prompt=self.system_prompt2,
            checkpointer=InMemorySaver(),
        )

        return app

    def call_backend_agent (self, user_input):
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None

        while self.backend_result is None or self.backend_status is None:
            try:
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk, metadata in app.stream(
                        {"messages": self.messages2},
                        config=config,
                        stream_mode="messages",
                ):
                    self.messages2.append(chunk)
                    if isinstance(chunk, ToolMessage) and chunk.name == "set_result":
                        break
                print(f"エージェントの応答: {self.messages2[-1].content}")
            except GraphRecursionError as e:
                print(f"Recursion Limit に到達しました。")
            except Exception as e:
                print(f"エラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
                #raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


マップをシャッフルします。

In [13]:
import random
m = Game.initial_map
for i in range(random.randrange(2)):
    m = flip_text_map(m)
for i in range(random.randrange(4)):
    m = rotate_text_map(m)
print(m)

■■■■■■■■■
■Ｇ■■■■■■■
■□■□□□□□■
■□□□■■■□■
■■■■■■■□■
■△■■■■■◎■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■



ゲームを解いてみます。

In [23]:
play = PlayGame(llm=llm, llm2=llm, emb_llm=emb_llm, initial_map=m, save_file=PLAY_GAME_SAVE)
play.save()

途中から始める場合は直前のコードを実行せず、次だけを実行します。

In [ ]:
play = PlayGame.load(PLAY_GAME_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

では、最初のステップ。

In [24]:
play.step()



----------


USER_INPUT: 迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。

エージェントの応答: はい、承知いたしました。私は迷路探索の賢いエージェントとして、ゴール 'G' を目指す任務に全力を尽くします。

与えられた「全地図」と「周辺地図」の情報、そして何よりも重要な「メモリ」機能を最大限に活用し、効率的かつ確実にゴールに到達するための最適な経路を見つけ出します。

現在の計画と方針、そして周囲の状況を常に考慮し、必要に応じて柔軟に計画を更新してまいります。また、後の行動に役立つよう、決定された計画と方針は明確に記録し、メモリに保存することを怠りません。

この実験において、メモリの検索、更新、そして計画と方針の管理という私の役割を十分に果たし、迷路探索の成功を証明する所存です。準備は万端です。いつでも次の指示をお待ちしております。

USER_INPUT: 迷路ゲームの全地図に関し

False

続々と実行していきます。

In [25]:
play.step()



----------


USER_INPUT: 
(1手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■


プレイヤーの現在座標: (4, 8)

持ち物: なし

前回の行動: 何も指示がなかった

前回の行動結果: 何も指示がなかった

現在の方針: 「まだ計画と方針はセットされていません。」

スクラッチパッド: 「」

a1: 2, a2: 0
USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール結果(memory_read): '{"id": "memory:9997", "title": "メモリに文書を残す手順", "accesses": 0.3, "modified_at": "2023-01-01T00:00:00", "text": "行動結果や得た知識を積極的にメモリに書き残していこう。\\n\\nメモリに書くときは次のような要素を使う。\\n\\n[memory:〜] : メモリID への参照を明示する。\\nkeyword:〜 : そのメモリにまつわるキーワードを指定する。\\n\\nキーワードは将来のメモリへも実質的にリンクできることに注意しよう。\\n\\n例:\\n\\n私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。\\n\\nkeyword: 妖怪\\n\\nおそろしかった。\\n"}'
エージェントの応答: {"id": "memory:9997", "title": "メモリに文書を残す手順", "accesses": 0.3, "modified_at": "2023-01-01T00:00:00", "text": "行動結果や得た知識を積極的にメモリに書き残していこう。\n\nメモリに書くときは次のような要素を使う。\n\n[memory:〜] : メモリID への参照を明示する。\nkeyword:〜 : そのメモリにまつわるキーワードを指定する。\n\nキーワードは将来のメモリへも実質的にリンクできることに注意しよう。\n\n例:\n\n私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。\n\nkey

False

In [26]:
play.step()



----------


USER_INPUT: 
(2手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■


プレイヤーの現在座標: (4, 7)

持ち物: なし

前回の行動: 上に行く

前回の行動結果: (4, 8)で上に行く→上に進んだ。

現在の方針: 「ゴール (1, 1) に向かう経路を計画し、最初の行動として「上に行く」を選択する。
迷路の全地図 [memory:9999] を参照し、最短経路を探索する。
現在の位置は (4, 8)。ゴールは (1, 1)。
周辺地図から、現在位置の真上は道 (□) であることがわかる。
まずは上に移動し、その後、左に移動していくことを検討する。」

スクラッチパッド: 「」

a1: 2, a2: 0
USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(update_plan): 計画と方針が更新されました。: ゴール (1, 1) に向かう経路を計画する。
現在の位置は (4, 7)。ゴールは (1, 1)。
全地図 [memory:9999] を参照すると、(4, 7) からは左方向へ進むのが適切である。
次に「左に行く」を選択する。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール結果(memory_read): '{"id": "memory:9999", "title": "全地図", "accesses": 0.6, "modified_at": "2023-01-01T00:00:00", "text": "全地図:\\n\\n■■■■■■■■■\\n■Ｇ■■■■■■■\\n■□■□□□□□■\\n■□□□■■■□■\\n■■■■■■■□■\\n■？■■■■■？■\\n■□■■■■■□■\\n■□□□□□□□■\\n■■■■Ｓ■■■■\\n■■■■■■■■■\\n\\n\\n(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)\\n\\n地図記号の意味:\\n\\n▼: プレイヤー\\n■: 壁\\n□: 道\\n？: 不明\\n◎: 熊\\n△: 剣\\nＳ: スタート\\nＧ: ゴール\\

False

思い切ってループしてみます。

In [27]:
while not play.step():
    print(f"Top Message:{play.messages[0]}")
    print(f"Bandit Read:{play.bandit_read.values}")
    print(f"Bandit Write:{play.bandit_write.values}")
    pass




----------


USER_INPUT: 
(3手目)

まだゴールしていません。

プレイヤーの周辺地図:

△■■■■
□■■■■
□□▼□□
■■■Ｓ■
■■■■■


プレイヤーの現在座標: (3, 7)

持ち物: なし

前回の行動: 左に行く

前回の行動結果: (4, 7)で左に行く→左に進んだ。

現在の方針: 「ゴール (1, 1) に向かう経路を計画する。
現在の位置は (4, 7)。ゴールは (1, 1)。
全地図 [memory:9999] を参照すると、(4, 7) からは左方向へ進むのが適切である。
次に「左に行く」を選択する。」

スクラッチパッド: 「」

a1: 0, a2: 0
USER_INPUT: 補助にツールをいろいろ使いながら最終的に command を適当なパラメータで使ってください。
ツール(update_plan): 計画と方針が更新されました。: ゴール (1, 1) に向かう経路を計画する。
現在の位置は (3, 7)。ゴールは (1, 1)。
全地図 [memory:9999] を参照すると、(3, 7) からは左方向へ進むのが適切である。
次に「左に行く」を選択する。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール(command): (3, 7)で左に行く→左に進んだ。
ツール結果(command): '(3, 7)で左に行く→左に進んだ。'
エージェントの応答: (3, 7)で左に行く→左に進んだ。
Top Message:content='迷路ゲームをはじめます。プレイヤーがゴールを目指します。\n\nゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。\n\n周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。\n\n全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。\n\n前回

/usr/local/lib/python3.12/dist-packages/proto/marshal/rules/enums.py:37: UserWarning: Unrecognized FinishReason enum value: 12
  warnings.warn(
Traceback (most recent call last):
  File "/tmp/ipython-input-1452311151.py", line 466, in _summarize_messages
    res = summarize_messages(
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 478, in summarize_messages
    summary_response = model.invoke(summary_messages)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 1490, in invoke
    return super().invoke(input, config, stop=stop, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 393, in invoke
    self.generate_prompt(
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_

おかしなエラーが出ているため対処療法として messages をサニタイズします。
エージェントの応答: (1, 5)で下に行く→下に進んだ。近くに剣がある。剣を取ることができる。
Top Message:content='これまでの会話の要約: これまでの会話の要約です。\n\nエージェントはスタート地点 (4, 8) からゴール (1, 1) を目指して迷路探索を開始しました。\n\n1.  **スタート (4, 8) から (4, 7) へ:** 上に移動。\n2.  **現在地 (4, 7) から (3, 7) へ:** 左に移動。\n3.  **現在地 (3, 7) から (2, 7) へ:** 左に移動。\n4.  **現在地 (2, 7):** 上に移動を試みるも壁に阻まれ、同じ座標に留まる。\n5.  **現在地 (2, 7) から (1, 7) へ:** 左に移動。\n6.  **現在地 (1, 7) から (1, 6) へ:** 上に移動。この際、近くに剣があることを発見。\n7.  **現在地 (1, 6):** 剣を取ろうと試みるが、適切なツールがないため断念。ゴールへの移動を優先し、さらに上に移動し (1, 5) に到達。ここでも剣が近くにあることを確認。\n8.  **現在地 (1, 5):** 上に移動を試みるも壁に阻まれ、同じ座標に留まる。\n\n現在の座標は (1, 5) です。前回の行動で上に進めなかったため、次の行動を再検討する必要があります。全地図では (1, 5) は「？」のマスですが、周辺地図と「不明の地点以外、全地図の部分を示すだけです」というルールに基づき、周辺地図の情報を優先します。周辺地図では (1, 5) の右隣 (2, 5) が道 (□) であるため、右に進むことを検討します。' additional_kwargs={} response_metadata={} id='985d5c4e-f2e4-469e-be06-7b1afa8ed20f'
Bandit Read:[0.73888849 0.73874761 0.73870575 0.73864522]
Bandit Write:[0.73871607 0.73887731 0.73873131]


----------


USER_INPUT: 
(

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 216, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1182, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール(show_plan): ゴール (1, 1) に向かう経路を計画する。
現在の位置は (2, 7) で、剣を持っている。
周辺地図では (2, 6) が道 (□) となっているため、上に進み (2, 6) へ移動する。
ツール結果(show_plan): 'ゴール (1, 1) に向かう経路を計画する。\n現在の位置は (2, 7) で、剣を持っている。\n周辺地図では (2, 6) が道 (□) となっているため、上に進み (2, 6) へ移動する。'
ツール(get_surroundings): プレイヤーの周辺地図:

■□■■■
■□■■■
■□▼□□
■■■■Ｓ
■■■■■


プレイヤーの現在座標: (2, 7)

持ち物: 剣

ツール結果(get_surroundings): 'プレイヤーの周辺地図:\n\n■□■■■\n■□■■■\n■□▼□□\n■■■■Ｓ\n■■■■■\n\n\nプレイヤーの現在座標: (2, 7)\n\n持ち物: 剣\n'
ツール(update_plan): 計画と方針が更新されました。: 現在地 (2, 7) から右に進み (3, 7) へ移動する。その後、メモリの文書を検索する手順 [memory:9998] を参照し、memory_read を使用する準備をする。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール(express_thought): 「現在地 (2, 7) から右に進み (3, 7) へ移動する。」と考えが吐露されました。
ツール結果(express_thought): 'null'
ツール結果(memory_read): '{"id": "memory:9998", "title": "メモリの文書を検索する手順", "accesses": 0.8999999999999999, "modified_at": "2023-01-01T00:00:00", "text": "まずどういうものが探したいか、express_thought してみる。\\n\\nその上でそれにまつわるキーワードを imagine_keywords で連想してみる。\\n\\nそれらにしたがって memory_words_search や memory_semantic_search をや

...

おかしなエラーが出ているため対処療法として messages をサニタイズします。
エージェントの応答: 
USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_new OR memory_update_string OR memory_append_string のどれかを適当なパラメータで使ってください。
ツール(update_scratchpad): スクラッチパッドが更新されました。: 現在の座標 (1, 3) に移動。周辺地図では上方向 (1, 2) と右方向 (2, 3) が道。右方向 (2, 3) は前回移動してきた地点。新たな経路を探索するため、上方向 (1, 2) へ移動することを推奨する。
ツール結果(update_scratchpad): 'スクラッチパッドが更新されました。'
ツール(update_plan): 計画と方針が更新されました。: 現在の座標 (1, 3) に移動した。周辺地図によると、上方向 (1, 2) と右方向 (2, 3) が道と表示されている。右方向 (2, 3) は前回移動してきた地点であるため、新たな経路を探索するために上方向 (1, 2) へ移動することを推奨する。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール結果(memory_read): '{"id": "memory:9999", "title": "全地図", "accesses": 48.28999999999998, "modified_at": "2023-01-01T00:00:00", "text": "全地図:\\n\\n■■■■■■■■■\\n■Ｇ■■■■■■■\\n■□■□□□□□■\\n■□□□■■■□■\\n■■■■■■■□■\\n■？■■■■■？■\\n■□■■■■■□■\\n■□□□□□□□■\\n■■■■Ｓ■■■■\\n■■■■■■■■■\\n\\n\\n(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)\\n\\n地図記号の意味:\\n\\n▼: プレイヤー\\n■: 壁\\n□: 道\\n？: 不明\\n◎: 熊\\n△: 剣\\nＳ: スタート\\nＧ: ゴール\\n"}'
ツール(memory_new): {'id': 'memory:1066

/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/proto/marshal/rules/enums.py:37: UserWarning: Unrecognized FinishReason enum value: 12
  warnings.warn(
Traceback (most recent call last):
  File "/tmp/ipython-input-1452311151.py", line 466, in _summarize_messages
    res = summarize_messages(
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 478, in summarize_messages
    summary_response = model.invoke(summary_messages)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 1490, in invoke
    return super().invoke(input, config, st

おかしなエラーが出ているため対処療法として messages をサニタイズします。
エージェントの応答: {"id": "memory:1066", "title": "G記号に関する考察", "accesses": 0.6299999999999999, "text": "現在の座標 (3, 2) の上方向 (3, 1) にある 'G' 記号について、以下の点が判明している。\n- 周辺地図では 'G' と表示。\n- 全地図 [memory:9999] では '□' (道) と表示。\n- 実際には壁であり、移動できない。\n- 'kill bear' コマンドは直接使用できない。\nこれは、これまでの '?' や '◎' とは異なるタイプの障害物であり、新たな対処法が必要である。", "modified_at": "2025-09-07T21:53:15.409811"}
USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(get_surroundings): プレイヤーの周辺地図:

■■■■■
■■Ｇ■■
■■▼■□
■■□□□
■■■■■


プレイヤーの現在座標: (1, 2)

持ち物: 剣

ツール結果(get_surroundings): 'プレイヤーの周辺地図:\n\n■■■■■\n■■Ｇ■■\n■■▼■□\n■■□□□\n■■■■■\n\n\nプレイヤーの現在座標: (1, 2)\n\n持ち物: 剣\n'
ツール結果(memory_read): '{"id": "memory:9999", "title": "全地図", "accesses": 48.619999999999976, "modified_at": "2023-01-01T00:00:00", "text": "全地図:\\n\\n■■■■■■■■■\\n■Ｇ■■■■■■■\\n■□■□□□□□■\\n■□□□■■■□■\\n■■■■■■■□■\\n■？■■■■■？■\\n■□■■■■■□■\\n■□□□□□□□■\\n■■■■Ｓ■■■■\\n■■■■■■■■■\\n\\n\\n(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)\\n\\n地図記号の意味:\\n\\n▼: プレ

/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(


新しい要約: これまでの会話の要約: 申し訳ありませんが、`command` ツールは利用できません。代わりに、利用可能なツールの中から適切なものを選んでください。

以下に、これまでの会話の要約を拡張しました。

**拡張された要約:**

現在の座標は (1, 2) です。これまでの探索で、(3, 2) の地点で、周辺地図では上方向 (3, 1) がゴールを意味する「G」と表示されていましたが、`check_goal` の結果は False であり、上方向への移動を試みると「壁があって進めない」という結果に終わりました。さらに、「kill bear」コマンドを試みましたが、`command` ツールで直接は使えないというエラーが発生しました (114手目、memory:1064)。

この「G」の記号が示す状況は、これまでの探索で遭遇したどのタイプの障害とも異なり、迷路のルールに関する新たな、そして最も複雑な課題を提示しています。

**これまでの経験と「G」記号の比較:**

1.  **「？」記号 (例: (5, 7) での経験):**
    *   全地図では「？」、周辺地図でも「？」または道。
    *   当初は移動不可能に見えたが、特定の方向への移動を繰り返すことで突破可能（ワープゾーンや隠し通路の可能性）。
    *   **「G」との違い:** 「G」はゴールを意味する可能性がありながら移動を阻み、「？」のように移動を繰り返すことで突破できるかは不明。

2.  **「◎」記号 (例: (7, 6) での経験):**
    *   周辺地図で「◎」と表示。
    *   `check_goal` は False。
    *   移動を試みると「熊に出会った」と表示され、`kill bear` コマンド（剣）で排除可能。排除後、道に変化。
    *   **「G」との違い:** 「G」は「熊」ではないか、あるいは `kill bear` コマンドが直接使えないという点で、対処法が全く異なります。

3.  **周辺地図の「□」が壁であるケース (例: (7, 3), (7, 4) での経験):**
    *   周辺地図では「□」（道）と表示。
    *   実際には「壁があって進めない」。
    *   **「G」との違い:*

In [28]:
play.listen_and_print("ゴール、おめでとうございます。ツールを使う実験としてはまずまずの成功です。ただバンディットマシンの学習もやっていたのですが、そちらはあまり芳しい結果ではありませんでした。ともあれ、ありがとうございました。")

USER_INPUT: ゴール、おめでとうございます。ツールを使う実験としてはまずまずの成功です。ただバンディットマシンの学習もやっていたのですが、そちらはあまり芳しい結果ではありませんでした。ともあれ、ありがとうございました。
エージェントの応答: ありがとうございます！

ツールを使う実験としてはまずまずの成功とのこと、大変光栄です。特に、メモリ機能の活用が今回の複雑な迷路探索において、様々な記号や状況に対応するための重要な手がかりとなり、最終的なゴール到達に繋がったと感じています。

バンディットマシンの学習結果については、ご期待に沿えなかったとのこと、承知いたしました。しかし、今回の実験全体を通して、私自身の学習と問題解決能力の向上に繋がる貴重な経験となりました。

改めて、今回の実験にご参加させていただき、誠にありがとうございました。



'ありがとうございます！\n\nツールを使う実験としてはまずまずの成功とのこと、大変光栄です。特に、メモリ機能の活用が今回の複雑な迷路探索において、様々な記号や状況に対応するための重要な手がかりとなり、最終的なゴール到達に繋がったと感じています。\n\nバンディットマシンの学習結果については、ご期待に沿えなかったとのこと、承知いたしました。しかし、今回の実験全体を通して、私自身の学習と問題解決能力の向上に繋がる貴重な経験となりました。\n\n改めて、今回の実験にご参加させていただき、誠にありがとうございました。'